# CME 257 Class 6 - Language Interfaces

One of the nice things about Julia is that it is relatively easy to use code written in other languages.  Today we'll talk about Julia's built-in [`ccall()`](http://julia.readthedocs.org/en/latest/manual/calling-c-and-fortran-code/) function (for C and Fortran) as well as the [PyCall](https://github.com/stevengj/PyCall.jl) package.  There are also packages to call [R](https://github.com/JuliaStats/RCall.jl), [Matlab](https://github.com/JuliaLang/MATLAB.jl), [Mathematica](https://github.com/one-more-minute/Mathematica.jl), [C++](https://github.com/Keno/Cxx.jl), and [Java](https://github.com/aviks/JavaCall.jl) (maybe more that aren't on the package registry).  We'll focus on ccall and PyCall today because these are probably the most important in the current Julia ecosystem, although you may find one of these other packages useful depending on your needs and interests.

## Why call other languages?

Julia can be nice to work with, but isn't perfectly suited for all problems, and hasn't been around for a long time to gain extensive package support.  Using language interfaces can let you

* Use Julia for certain tasks, and use a different language for other tasks
* Utilize robust, tried and tested libraries, or industry/community standard libraries
* Use code from your old projects in Julia

## ccall

`ccall()` lets you call libraries written in either in c or fortran from Julia.  Shared object libraries vary a bit between operating systems - typically on linux they have a .so extension, Macs have a .dylib extension, and Windows has a .dll extension.  Static libraries have a .a extension on Mac/linux, and .lib extension on Windows.  The examples here were tested on linux machines (Elementary OS, Fedora, and Ubuntu), so if you may find it easier to follow along on corn@stanford.

Shared object libraries are loaded at runtime, and static libraries have code that is copied when a binary is created.  You can only call shared object libraries from Julia.  If you want to learn more, try this [StackOverflow thread](http://stackoverflow.com/questions/2649334/difference-between-static-and-shared-libraries) to start.

### Compiling a library

When you install a library using a package manager, it's pretty easy to get started using it.  If you have your own custom code, you need to pass in `-fPIC` and `-shared` to your compiler (at least gcc, other compilers may behave slightly differently) to help the complier know it is creating a shared object library.

Here we'll use C to compile libraries, but you can also use fortran.

Refer to the [Makefile](Makefile) to see a basic example.

### Using ccall

Refer to [cme257.c](cme257.c) to see the function declarations.

In [ ]:
function c_hello()
    ccall((:hello, "libcme257"), Void, ())
end

function c_sum(a::Int64, b::Int64)
    return ccall((:c_sum, "libcme257"), Int64, (Float64, Float64), Float64(a), Float64(b))
end
;

In [ ]:
@show c_sum(100, 5)
c_hello()

ccall is used in parts of Julia, and also in some common libraries.

* [Metis.jl](https://github.com/JuliaSparse/Metis.jl) is simply a wrapper for the Metis library (graph partioning). (Other packages do similar things).
* Julia allows you to direcly call [BLAS](http://www.netlib.org/blas/) as a [feature of the language](https://github.com/JuliaLang/julia/blob/master/base/linalg/blas.jl).

For example,

In [ ]:
using Base.Test

begin
n = 1000;
A = randn(n, n);
B = randn(n, n);
C = randn(n, n);
Corig = deepcopy(C);
α = 2.0;
β = 3.0;

# C = α A B' + β C
@time BLAS.gemm!('N', 'T', α, A, B, β, C); #@time to time

@show C ≈ (α * A * B' + β * Corig)

C = copy(Corig);

@time C = α * A * B' + β * C;

end



# Exercise 1

* modify the cme257.c to include a multiplication function.  Create a function in Julia that calls the multiplication function in the shared object library.
* Compare the time it takes to do GEMM calling BLAS versus writing out the corresponding expression in Julia (try n = 10, 100, 1000).  Can you explain the results?

# PyCall

[PyCall](https://github.com/stevengj/PyCall.jl) is a package by [Stephen Johnson](http://math.mit.edu/~stevenj/), which allows you to call Python libraries using syntax that is essentially the same as Python's import statement.

It is used in several packages in Julia, including PyPlot.

In [ ]:
using PyCall

In [ ]:
# Demo similar to https://github.com/stevengj/PyCall.jl#usage
@pyimport math
x = 5;
math.sin(math.pi + x) - sin(π + x)

PyCall works by combining Python's C API, and Julia's `ccall()` functions (so you can't use Jython, unless you want to try [JavaCall](https://github.com/aviks/JavaCall.jl)).  Check out the [source for PyCall](https://github.com/stevengj/PyCall.jl/blob/master/src/PyCall.jl), and see how `ccall` is being used.

Here's an example using Python's popular [scikit-learn](http://scikit-learn.org/stable/) from Julia:

In [ ]:
# https://rizalzaf.wordpress.com/2015/05/15/calling-pythons-scikit-learn-machine-learning-library-from-julia/
@pyimport sklearn.svm as svm
X = [[0 0]; [1 1]]
y = [0; 1]
clf = svm.SVC()
clf[:fit](X, y)

In [ ]:
x_test = [0.01 0.02]
y_test = clf[:predict](x_test)

You can also import your own Python modules.  Refer to [cme257.py](cme257.py). This is currently a [little hacky](https://github.com/stevengj/PyCall.jl/issues/48) if your module is not on Python's path.

In [ ]:
# modified from https://github.com/dhoegh/Hawk.jl/blob/master/src/Hawk.jl#L7-13
# there is a if you are using python version 3, use importlib.machinery instead
# I'm using python 2.7
filename = abspath(joinpath(dirname(@__FILE__),"cme257.py"))
@pyimport imp
(path, name) = dirname(filename), basename(filename)
(name, ext) = rsplit(name, '.', limit=2)
(file, filename, data) = imp.find_module(name, [path])
cme257 = imp.load_module(name, file, filename, data)

for i = 0:10
    println("fibonacci($i) = $(cme257[:fibonacci](i))")
end

## Exercise 2

(You may need to install scikit learn to do this - `conda install scikit-learn` may do the trick if you have Anaconda).
* use a [decision tree classifier](http://scikit-learn.org/stable/modules/tree.html#classification) from scikit-learn on the example above.
* modify cme257.py to include a function that adds 3 integers together and call it from Julia
* How long does it take to multiply two 100x100 matrices using numpy?  How long does it take if you call numpy from Julia? How does this compare to doing the same thing in native Julia?